## Analyzing Large Datasets with Pandas

Pandas simplifies the analysis of large datasets. In this section, we'll use the database of passengers on the *Titanic*, available through the Seaborn library, to learn about some of Pandan's capabilities. 


The Titanic dataset is a well-known dataset that contains information such as passengers' gender, age, class, fare paid, survival status, and much more. 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [ ]:
# You can take a peek at the start of the data with the .head() function. This action allows you to see the columns and type of data you are dealing with
titanic.head()

- pclass: Passenger Class
- sibsp: Number of siblings/spouses aboard the Titanic

In [ ]:
#you can take a peek at the end of the data with the .tail() function
titanic.tail()

In [ ]:
#you obtain a quick summary of the numeric columns in your data using .describe() 
titanic.describe()

## Pivot Tables by Hand

To start learning more about this data, we might begin by grouping according to gender, survival status, or some combination thereof.
You can use the ``GroupBy`` operation–for example, let's look at the survival rate by gender:

In [ ]:
titanic.groupby('sex')[['survived']].mean()

This immediately gives us some insight: overall, three of every four females on board survived, while only one in five males survived!

This is useful, but we might like to go one step deeper and look at survival by both sex and, say, class.
Using the vocabulary of ``GroupBy``, we might proceed using something like this:
we *group by* class and gender, *select* survival, *apply* a mean aggregate, *combine* the resulting groups, and then *unstack* the hierarchical index to reveal the hidden multidimensionality. In code:

In [ ]:
titanic.groupby(['sex', 'class'], observed=True)['survived'].mean().unstack()

This gives us a better idea of how both gender and class affected survival, but the code is starting to look a bit garbled.
While each step of this pipeline makes sense in light of the tools we've previously discussed, the long string of code is not particularly easy to read or use.
This two-dimensional ``GroupBy`` is common enough that Pandas includes a convenience routine, ``pivot_table``, which succinctly handles this type of multi-dimensional aggregation.

## Pivot Table Syntax

Here is the equivalent to the preceding operation using the ``pivot_table`` method of ``DataFrame``:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', observed=True)

This is eminently more readable than the ``groupby`` approach, and produces the same result.


### Multi-level pivot tables

Just as in the ``GroupBy``, the grouping in pivot tables can be specified with multiple levels, and via a number of options.
For example, we might be interested in looking at age as a third dimension.

We'll bin the age using the ``pd.cut`` function, which identifies the age range of each passenger, [0, 18] or (18,80]

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])
age

In [ ]:
titanic.pivot_table('survived', ['sex', age], 'class', observed=True)

We can apply the same strategy when working with the columns as well; let's add info on the fare paid using ``pd.qcut`` to automatically compute quantiles:

In [ ]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'], observed=True)

The result is a four-dimensional aggregation with hierarchical indices, shown in a grid demonstrating the relationship between the values.

### Additional pivot table options

The full call signature of the ``pivot_table`` method of ``DataFrame``s is as follows:

```python
# call signature as of Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```

We've already seen examples of the first three arguments; here we'll take a quick look at the remaining ones.
Two of the options, ``fill_value`` and ``dropna``, have to do with missing data and are fairly straightforward; we will not show examples of them here.

The ``aggfunc`` keyword controls what type of aggregation is applied, which is a mean by default.
As in the GroupBy, the aggregation specification can be a string representing one of several common choices (e.g., ``'sum'``, ``'mean'``, ``'count'``, ``'min'``, ``'max'``, etc.) or a function that implements an aggregation (e.g., ``np.sum()``, ``min()``, ``sum()``, etc.).
Additionally, it can be specified as a dictionary mapping a column to any of the above desired options:

In [ ]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':'sum', 'fare':'mean'}, observed=True)

Notice also here that we've omitted the ``values`` keyword; when specifying a mapping for ``aggfunc``, this is determined automatically.

At times it's useful to compute totals along each grouping.
This can be done via the ``margins`` keyword:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True, observed=True)

Here this automatically gives us information about the class-agnostic survival rate by gender, the gender-agnostic survival rate by class, and the overall survival rate of 38%.
The margin label can be specified with the ``margins_name`` keyword, which defaults to ``"All"``.

## Exercise

Next, you will use your Pandas skills to analyze a new dataset. The data is related to births in the United States, provided by the Centers for Disease Control (CDC). 

This data has been studied extensively, and there are several websites describing methods for its analysis. 

Try to do the following analysis yourself. If you get stuck, the full solution is at the end of the notebook. 

In [ ]:
# Let's start by reading the data and looking at it
births_data = pd.read_csv('data/births.csv', index_col = None)

This data contains many outliers caused by typos. One way to remove outliers is to use sigma clipping, which uses the 25th, 50th, and 75th percentiles of the data. The methods can be used as follows:

```Python
quartiles = np.percentile(births_data['births'],[25,50,75]) # compute the 25th, 50th, and 75th percentiles
mu = quartiles[1] #get the 50th percentile (median)
sigma = 0.74 * (quartiles[2] - quartiles[0]) # compute the sigma threshold
births_data = births_data.query('(births > @mu - 5 * @sigma) & (births < @mu + 5 * @sigma)')
```

In [ ]:
#write your code to eliminate outliers here


In [ ]:
#print the head of the data to get an idea of what it contains

#write your code here


In [ ]:
#print the tail of the data to get an idea of what it contains

#write your code here


as you can see, the data contains the number of births (both Male (M) and Female (F)) from 1969 until 1988. Some of the data is arranged by days, and other is by months. 

In [ ]:
#summarize the total number of births per year (both M and F) using a pivot table 
#write your code here


This large table is difficult to read, create a visual representation of the data by adding ```.plot()``` after you create your pivot_table

In [ ]:
#write your code here


Using this plot, is easy to see that the has been more male birth than female births in the US. And that the number of births shows a growing trend. 

Try it yourself. If you get stuck, the solution is at the end of the notebook. 

|

|

|

|

|

|

|

|

|

|

|

|

|

|

Are you sure you want to see the answer?

|

|

|

|

|

|

|

|

|

|

|

|

|

|

ok....

In [ ]:
#write your code to eliminate outliers here
quartiles = np.percentile(births_data['births'],[25,50,75]) # compute the 25th, 50th, and 75th percentiles
mu = quartiles[1] #get the 50th percentile (median)
sigma = 0.74 * (quartiles[2] - quartiles[0]) # compute the sigma threshold
births_data = births_data.query('(births > @mu - 5 * @sigma) & (births < @mu + 5 * @sigma)')

In [ ]:
#print the head of the data to get an idea of what it contains

#write your code here
births_data.head()

In [ ]:
#print the tailof the data to get an idea of what it contains

#write your code here
births_data.tail()

In [ ]:
#summarize the total number of births per year (both M and F) using a table 

#write your code here
births_data.pivot_table('births', index='year', columns = 'gender', aggfunc = 'sum')

In [ ]:
#write your code here
births_data.pivot_table('births', index='year', columns = 'gender', aggfunc = 'sum').plot()